In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_40.csv')
pred2 = dd.read_csv('test_pred_41.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 33 s, sys: 3.12 s, total: 36.1 s
Wall time: 7.61 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 222 ms, sys: 244 ms, total: 466 ms
Wall time: 464 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 766 µs, sys: 97 µs, total: 863 µs
Wall time: 878 µs


In [7]:
coef_lgb = 0.7
coef_mlp = 0.3

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 336 ms, sys: 541 ms, total: 877 ms
Wall time: 873 ms


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [10]:
test_ensemble['object_id'] = pred1['object_id']
print(test_ensemble.shape)

(3492890, 16)


In [11]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [12]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000151,0.357796,0.000308,0.198685,0.115433,0.000352,0.109848,0.000250,0.001310,0.014863,0.002204,0.198082,0.000227,0.000488,0.246366
3492886,130787971,0.000165,0.014352,0.000159,0.058426,0.180788,0.000197,0.022953,0.322159,0.000324,0.036385,0.000748,0.357466,0.000261,0.005617,0.231201
3492887,130787974,0.000704,0.021592,0.000124,0.831549,0.029592,0.000301,0.102202,0.000043,0.000126,0.001323,0.000318,0.010166,0.000166,0.001792,0.109838
3492888,130788053,0.000053,0.740299,0.000067,0.141608,0.003562,0.000103,0.010289,0.072028,0.000408,0.000399,0.000258,0.030113,0.000097,0.000715,0.154847
3492889,130788054,0.000172,0.077831,0.000253,0.396815,0.192522,0.000417,0.053251,0.000109,0.000455,0.020092,0.001007,0.255794,0.000556,0.000725,0.236729


In [13]:
%%time
test_ensemble.to_csv('test_pred_48.csv',index=False)

CPU times: user 1min 31s, sys: 1.17 s, total: 1min 32s
Wall time: 1min 32s


In [ ]:
test_ensemble['object_id'].equals(pred1['object_id'])

In [ ]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_48.csv -m "Ensemble_LGB_MLP_07_03"